# Recurrent Neural Network(再帰型ニューラルネットワーク)

 - [EC購買サンプルデータセット](https://support.treasuredata.com/hc/ja/articles/213697057-%E3%82%B5%E3%83%B3%E3%83%97%E3%83%AB%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88-EC-)

## 問題 : sin波の予測
時刻$1, \dots, t$までのノイズ入りsin波の値$f\left(1\right), \dots, f\left(t\right)$が与えらたとき、時刻$t+1$の値$f\left( t + 1 \right)$を予測する

In [19]:
def sin(x, T=100):
    """sin関数"""
    return np.sin(2.0 * np.pi * x / T)

def toy_problem(T=100, ampl=0.05):
    """サイン波 + ノイズのデータを生成する"""
    x = np.arange(0, 2 * T + 1)
    noise = ampl * np.random.uniform(low=-1.0, high=1.0, size=len(x))
    return sin(x) + noise

In [20]:
T = 100
f = toy_problem(T)
f.shape

(201,)

### 時系列データの準備

In [21]:
length_of_sequences = 2 * T    # 全時系列の長さ
maxlen = 25    # 1つの時系列データの長さ


data = []
target = []

for i in range(0, length_of_sequences - maxlen + 1):
    data.append(f[i: i + maxlen])
    target.append(f[i + maxlen])

In [22]:
X = np.array(data).reshape(len(data), maxlen, 1)
Y = np.array(target).reshape(len(data), 1)

In [ ]:
N_train = int(len(data) * 0.9)
N_test = len(data) - N_train

X_train, X_test, Y_train, Y_test = train